# <center> Analysis of input data and constraints of columns </center>

In [65]:
print("Dataset:", (len("Dominik") + len("Zurek")) % 6)

Dataset: 0


## Basic data analysis and cleanup

In [66]:
import pandas as pd

In [67]:
df = pd.read_csv("houses_to_rent.csv", index_col=0)
df.head()

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa,rent amount,property tax,fire insurance,total
0,1,240,3,3,4,-,acept,furnished,R$0,"R$8,000","R$1,000",R$121,"R$9,121"
1,0,64,2,1,1,10,acept,not furnished,R$540,R$820,R$122,R$11,"R$1,493"
2,1,443,5,5,4,3,acept,furnished,"R$4,172","R$7,000","R$1,417",R$89,"R$12,680"
3,1,73,2,2,1,12,acept,not furnished,R$700,"R$1,250",R$150,R$16,"R$2,116"
4,1,19,1,1,0,-,not acept,not furnished,R$0,"R$1,200",R$41,R$16,"R$1,257"


In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6080 entries, 0 to 6079
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   city            6080 non-null   int64 
 1   area            6080 non-null   int64 
 2   rooms           6080 non-null   int64 
 3   bathroom        6080 non-null   int64 
 4   parking spaces  6080 non-null   int64 
 5   floor           6080 non-null   object
 6   animal          6080 non-null   object
 7   furniture       6080 non-null   object
 8   hoa             6080 non-null   object
 9   rent amount     6080 non-null   object
 10  property tax    6080 non-null   object
 11  fire insurance  6080 non-null   object
 12  total           6080 non-null   object
dtypes: int64(5), object(8)
memory usage: 665.0+ KB


In [69]:
rial_columns = ["hoa", "rent amount", "property tax", "fire insurance", "total"]


def filter_non_rial_entries(df: pd.DataFrame, rial_columns: list[str]) -> pd.DataFrame:
    rial_regex = r"R\$\w*\,?\w*"

    base = df[rial_columns[0]].str.contains(rial_regex, na=False)
    for col in rial_columns[1:]:
        base *= df[col].str.contains(rial_regex, na=False)

    return df[base].reset_index(drop=True)


def convert_rials_to_integers(
    df: pd.DataFrame, rial_columns: list[str]
) -> pd.DataFrame:
    rial_regex = "R\$(\w+)"
    for col in rial_columns:
        df[col] = df[col].str.replace(",", "").str.extract(rial_regex).astype(int)
    return df


filtered_non_rials = filter_non_rial_entries(df, rial_columns)
cleaned_rials = convert_rials_to_integers(filtered_non_rials, rial_columns)
cleaned_rials.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5852 entries, 0 to 5851
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   city            5852 non-null   int64 
 1   area            5852 non-null   int64 
 2   rooms           5852 non-null   int64 
 3   bathroom        5852 non-null   int64 
 4   parking spaces  5852 non-null   int64 
 5   floor           5852 non-null   object
 6   animal          5852 non-null   object
 7   furniture       5852 non-null   object
 8   hoa             5852 non-null   int64 
 9   rent amount     5852 non-null   int64 
 10  property tax    5852 non-null   int64 
 11  fire insurance  5852 non-null   int64 
 12  total           5852 non-null   int64 
dtypes: int64(10), object(3)
memory usage: 594.5+ KB


In [70]:
cleaned_rials["pet_friendly"] = cleaned_rials.animal.map(
    {"acept": True, "not acept": False}
).astype("bool")
cleaned_rials["furnished"] = cleaned_rials.furniture.map(
    {"furnished": True, "not furnished": False}
).astype("bool")
cleaned_rials["floor"] = cleaned_rials.floor.str.replace("-", "0").astype(int)

cleaned_rials.drop(["animal", "furniture"], axis=1, inplace=True)
cleaned_rials.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5852 entries, 0 to 5851
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   city            5852 non-null   int64
 1   area            5852 non-null   int64
 2   rooms           5852 non-null   int64
 3   bathroom        5852 non-null   int64
 4   parking spaces  5852 non-null   int64
 5   floor           5852 non-null   int64
 6   hoa             5852 non-null   int64
 7   rent amount     5852 non-null   int64
 8   property tax    5852 non-null   int64
 9   fire insurance  5852 non-null   int64
 10  total           5852 non-null   int64
 11  pet_friendly    5852 non-null   bool 
 12  furnished       5852 non-null   bool 
dtypes: bool(2), int64(11)
memory usage: 514.5 KB


In [71]:
cleaned_rials.head()

,city,area,rooms,bathroom,parking spaces,floor,hoa,rent amount,property tax,fire insurance,total,pet_friendly,furnished
0,1,240,3,3,4,0,0,8000,1000,121,9121,True,True
1,0,64,2,1,1,10,540,820,122,11,1493,True,False
2,1,443,5,5,4,3,4172,7000,1417,89,12680,True,True
3,1,73,2,2,1,12,700,1250,150,16,2116,True,False
4,1,19,1,1,0,0,0,1200,41,16,1257,False,False


## Database structure

![ERD Diagram](erd.png)

In [97]:
import os
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base

db_string = os.environ.get("ADB_DB_STRING")

engine = create_engine(db_string)

Base = declarative_base()


from sqlalchemy import (
    Column,
    Integer,
    Boolean,
    ForeignKey,
    Sequence,
    CheckConstraint,
    UniqueConstraint,
)


class City(Base):
    __tablename__ = "cities"
    __table_args__ = (UniqueConstraint("city"),)
    id = Column(Integer, Sequence("seq_city_id"), primary_key=True)
    city = Column(Integer, nullable=False)


class HouseData(Base):
    __tablename__ = "house_data"
    __table_args__ = (
        CheckConstraint("area > 0"),
        CheckConstraint("rooms > 0"),
        CheckConstraint("bathrooms > 0"),
        CheckConstraint("parking_spaces >= 0"),
        CheckConstraint("floor >= 0"),
    )
    id = Column(Integer, Sequence("seq_house_data_id"), primary_key=True)
    city_id = Column(Integer, ForeignKey("cities.id"))
    area = Column(Integer, nullable=False)
    rooms = Column(Integer, nullable=False)
    bathrooms = Column(Integer, nullable=False)
    parking_spaces = Column(Integer, nullable=False)
    floor = Column(Integer, nullable=False)
    pet_friendly = Column(Boolean, nullable=False)
    furnished = Column(Boolean, nullable=False)


class RentData(Base):
    __tablename__ = "rent_data"
    __table_args__ = (
        CheckConstraint("hoa >= 0"),
        CheckConstraint("rent_amount >= 0"),
        CheckConstraint("property_tax >= 0"),
        CheckConstraint("fire_insurance >= 0"),
    )
    id = Column(Integer, Sequence("seq_rent_data_id"), primary_key=True)
    house_id = Column(Integer, ForeignKey("house_data.id"))
    hoa = Column(Integer, nullable=False)
    rent_amount = Column(Integer, nullable=False)
    property_tax = Column(Integer, nullable=False)
    fire_insurance = Column(Integer, nullable=False)
    total = Column(Integer, nullable=False)


Base.metadata.create_all(engine)

/var/folders/bd/sqft7pw95xx5vb3vsgj_30tm0000gn/T/ipykernel_7649/2882558642.py:9: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


## Prepare data for insertion

In [80]:
cities = pd.DataFrame(cleaned_rials["city"].unique(), columns=["city"])
cities.index.name = "id"
cities

,city
id,
0,1
1,0


In [81]:
house_data = cleaned_rials[
    [
        "area",
        "rooms",
        "bathroom",
        "parking spaces",
        "floor",
        "city",
        "furnished",
        "pet_friendly",
    ]
]
house_data.index.name = "id"
house_data = house_data.rename(
    columns={
        "city": "city_id",
        "bathroom": "bathrooms",
        "parking spaces": "parking_spaces",
    }
)
house_data["city_id"] = house_data["city_id"].map(
    lambda x: cities[cities["city"] == x].index.values.astype(int)[0]
)
house_data.head()

,area,rooms,bathrooms,parking_spaces,floor,city_id,furnished,pet_friendly
id,,,,,,,,
0,240,3,3,4,0,0,True,True
1,64,2,1,1,10,1,False,True
2,443,5,5,4,3,0,True,True
3,73,2,2,1,12,0,False,True
4,19,1,1,0,0,0,False,False


In [86]:
rent_data = cleaned_rials[rial_columns]
rent_data = rent_data.rename(
    columns={
        "rent amount": "rent_amount",
        "property tax": "property_tax",
        "fire insurance": "fire_insurance",
    }
)
rent_data["house_id"] = house_data.index
rent_data.head()

,hoa,rent_amount,property_tax,fire_insurance,total,house_id
id,,,,,,
0,0,8000,1000,121,9121,0
1,540,820,122,11,1493,1
2,4172,7000,1417,89,12680,2
3,700,1250,150,16,2116,3
4,0,1200,41,16,1257,4


## Insert data

In [98]:
cities.to_sql("cities", engine, if_exists="append")
house_data.to_sql("house_data", engine, if_exists="append")
rent_data.to_sql("rent_data", engine, if_exists="append")

852